In [ ]:
## this block will conduct t-tests on the emotion label PCs
## based on the ROI (MD or amygdala), Holm corrected
## and return a graph showing the mean of peak and trough 
## of the ROI and whether the difference is significant

## NOTE: you'll have to use the "emotion_labels__PCA_and_inter-rater_reliability_git"
## to get the "portrayed_PC_Scores_labeled_by_movie" csv and the viewer csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

## do the stats (t-test with holm-correction)

# 1. Read data
df = pd.read_csv("portrayed_PC_scores_labeled_by_movie.csv")

# 2. Define contrasts and mappings
comparisons = [("AM_high", "AM_low"), ("MD_high", "MD_low")]
label_map = {
    "AM_high": "amygdala peak",
    "AM_low":  "amygdala trough",
    "MD_high": "MD peak",
    "MD_low":  "MD trough"
}
colors = {"AM": "#EDC001", "MD": "#A5B299"}

# 3. Run t-tests and collect p-values
results = []
raw_p = []
for hi, lo in comparisons:
    subset = df[df["label"].isin([hi, lo])]
    desc = subset.groupby("label")["PC1"].agg(["mean", "sem"]).reindex([hi, lo])
    t_stat, p_val = ttest_ind(
        subset[subset["label"] == hi]["PC1"],
        subset[subset["label"] == lo]["PC1"],
        equal_var=True, nan_policy="omit"
    )
    n1 = subset[subset["label"] == hi].shape[0]
    n2 = subset[subset["label"] == lo].shape[0]
    dfree = n1 + n2 - 2
    results.append({
        "hi": hi,
        "lo": lo,
        "desc": desc,
        "t": t_stat,
        "p_raw": p_val,
        "df": dfree
    })
    raw_p.append(p_val)

# 4. Holm correction
_, p_corr, _, _ = multipletests(raw_p, method="holm")
for r, pc in zip(results, p_corr):
    r["p_corr"] = pc

# 5. Print summary table
print("Contrast\tMean_hi±SEM\tMean_lo±SEM\tt-stat\t df\tp_corr")
for r in results:
    hi, lo = r["hi"], r["lo"]
    desc, t_stat, p_corr, dfree = r["desc"], r["t"], r["p_corr"], r["df"]
    mean_hi, sem_hi = desc.loc[hi, "mean"], desc.loc[hi, "sem"]
    mean_lo, sem_lo = desc.loc[lo, "mean"], desc.loc[lo, "sem"]
    print(f"{hi} vs {lo}\t"
          f"{mean_hi:.3f}±{sem_hi:.3f}\t"
          f"{mean_lo:.3f}±{sem_lo:.3f}\t"
          f"{t_stat:.3f}\t"
          f"{dfree}\t"
          f"{p_corr:.4g}")

# 6. Significance stars helper
def stars(p):
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    return "ns"

## now make the graphs

# 1. Create figure with constrained layout
fig, axes = plt.subplots(1, 2, figsize=(14, 7), constrained_layout=True)

# 2. Plot each panel, with extra breathing room
for ax, r in zip(axes, results):
    hi, lo = r["hi"], r["lo"]
    desc, pc = r["desc"], r["p_corr"]
    grp = hi.split("_")[0]
    x = np.arange(2)

    ax.bar(
        x, desc["mean"], yerr=desc["sem"],
        width=0.6,
        color=colors[grp],
        edgecolor="black",
        capsize=5
    )
    ax.axhline(0, color="gray", linestyle="--", linewidth=1)

    # bracket & stars
    y_top = (desc["mean"] + desc["sem"]).max()
    y_bot = (desc["mean"] - desc["sem"]).min()
    bracket_y = y_top + 0.05 * (y_top - y_bot)
    h = 0.02 * (y_top - y_bot)
    ax.plot([0, 0, 1, 1],
            [bracket_y, bracket_y+h, bracket_y+h, bracket_y],
            color="black", lw=1.2)
    ax.text(0.5, bracket_y + h*1.2,
            stars(pc),
            ha="center", va="bottom", fontsize=12)

    # expand y-limits
    y_range = y_top - y_bot
    ax.set_ylim(y_bot - 0.1*y_range, bracket_y + 0.3*y_range)

    # labels & title
    ax.set_xticks(x)
    ax.set_xticklabels([label_map[hi], label_map[lo]], fontsize=12)
    ax.set_ylabel("Mean PC1", fontsize=14)
    ax.set_xlabel("Group", fontsize=14)
    region = "Amygdala" if grp == "AM" else "MD"
    ax.set_title(
        f"Mean Portrayed PC1 ± SEM: {region} Peak vs. Trough\n"
        f"(p = {pc:.4g}, Holm-corrected)",
        fontsize=16
    )

# 3. Panel labels
axes[0].text(-0.12, 1.10, "a", transform=axes[0].transAxes, fontsize=16)
axes[1].text(-0.12, 1.10, "b", transform=axes[1].transAxes, fontsize=16)

# 4. Significance key (moved below x-axis)
fig.text(
    0.5, -0.04,
    "ns: p ≥ 0.05    *: p < 0.05    **: p < 0.01    ***: p < 0.001",
    ha="center", fontsize=12
)

plt.show()
